# How meaningful are Gaussian ODE filter uncertainties?

We investigate whether the uncertaintes returned by Gaussian ODE Filters are meaningful as actual error estimates---or even better?


In [1]:
# Make inline plots vector graphics instead of raster graphics
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf', 'svg')

# Plotting
import matplotlib.pyplot as plt
#plt.rcParams['figure.figsize'] = [9.5, 6]
plt.rcParams['font.size'] = 18 
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsfonts}', r'\usepackage{amsmath}']

In [2]:
import matplotlib.pyplot as plt
import numpy as np

from diffeq.randomvariable import gaussian
from diffeq.solver import ode, odefilter

ModuleNotFoundError: No module named 'diffeq'

We start with the Lotka-Volterra equations

In [ ]:
initdist = gaussian.MultivariateGaussian(np.array([20., 20.]), np.eye(2)*0.0)

lv = ode.lotkavolterra([0., 50.], initdist)

The Kalman ODE filter is quite fast and flexible, but does not yield accurate uncertainty estimates. We see below that the uncertainty increases monotonously, independent on the actual peaks and valleys of the ODE solution.

It is not surprising that the Kalman solution is agnostic of the trajectory. In fact, the covariance of the Kalman filter is independent of the data and as such, we cannot expect it to return useful uncertainty.

In [ ]:
ms, cs, ts = odefilter.solve_filter_h(lv, 0.001, which_filt="kf", which_prior="ibm1")

In [ ]:
std1 = np.sqrt(cs[:, 0, 0])
std2 = np.sqrt(cs[:, 2, 2])

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True)
ax1.plot(ts, ms[:, 0], label="x1")
ax1.plot(ts, ms[:, 2], label="x2")
ax1.legend()
ax2.fill_between(ts,std1, alpha=0.25, label="x1-unc.")
ax2.fill_between(ts,std2, alpha=0.25, label="x2-unc.")
ax2.legend()
fig.suptitle("Kalman Filter")
plt.show()

If instead of the Kalman filter, we use the Extended Kalman filter, the uncertainties are not independent of the measurements (information about $f$ enters the covariance via $\nabla f$ in the measurement model). We can see that in the uncertainty plot below.

In [ ]:
ms, cs, ts = odefilter.solve_filter_h(lv, 0.001, which_filt="ekf", which_prior="ibm1")

In [ ]:
std1 = np.sqrt(cs[:, 0, 0])
std2 = np.sqrt(cs[:, 2, 2])

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True)
ax1.plot(ts, ms[:, 0], label="x1")
ax1.plot(ts, ms[:, 2], label="x2")
ax1.legend()
ax2.fill_between(ts,std1, alpha=0.25, label="x1-unc.")
ax2.fill_between(ts,std2, alpha=0.25, label="x2-unc.")
ax2.legend()
fig.suptitle("Extended Kalman Filter")
plt.show()

Notice that the uncertainties are aware of the peaks and valleys. They even know whether the peak is flat (rhs of blue, lhs of orange; smaller ones of the blue peaks) or steep. On top of that, they increase over time.

In [ ]:
initdist = gaussian.MultivariateGaussian(np.array([20., 20.]), np.eye(2)*0.0)

lv = ode.lotkavolterra([0., 37.], initdist)
ms, cs, ts = odefilter.solve_filter_h(lv, 0.001, which_filt="ekf", which_prior="ibm1")

In [ ]:
# This is for the IMPRS talk

plt.rcParams.update({'font.size': 22})


std1 = np.sqrt(cs[:, 0, 0])
std2 = np.sqrt(cs[:, 2, 2])

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(16,9), sharex=True)
ax1.plot(ts, ms[:, 0], label="x1", linewidth=4)
ax1.plot(ts, ms[:, 2], label="x2", linewidth=4)
ax1.legend()
ax1.set_ylabel("Trajectory")
ax2.fill_between(ts,std1, alpha=0.25, label="x1-unc.")
ax2.fill_between(ts,std2, alpha=0.25, label="x2-unc.")
ax2.plot(ts, std1, alpha=0.5, linewidth=2)
ax2.plot(ts, std2, alpha=0.5, linewidth=2)
ax2.set_xlabel("Time T")
ax2.set_ylabel("Uncertainty")
ax2.legend()
plt.savefig("uncert_lv", dpi=300)
plt.show()